<h1 align="center">
    <img 
        src="./img/Microsoft-Logo.png" 
        width="400"/>
</h1>
<h1 align="center">
    <b>Practical Guide</b>
</h1>
<h4 align="center">
    for the creation of an AI Solution using an accelerator from the <a href="https://www.ds-toolkit.com/">Data Science Toolkit</a>
</h4>

# What to expect

* **Challenge 1:** *Create your own AI solution*
* **Challenge 2:** *Getting answers from our new Copilot*
* **Challenge 3:** *Evaluate the quality of the AI solution*

# Challenge 1: *Create your own AI solution*

Here we are going to create a RAG based Copilot to answer questions about 6 sustainability reports from Microsoft, Apple, Amazon, Google, Meta and Netflix from 2021 or 2022. The documents are part of the  [Mini Esg Bench Dataset](https://llamahub.ai/l/llama_datasets/Mini%20ESG%20Bench%20Dataset?from=llama_datasets).

## Challenge 1 - Step 1:  *Let's import the libraries to be used in this notebook*.

> This step will take around **30 to 90 seconds** to complete. It is going to be done in a quiet mode, so only errors will be displayed (if they occur). If you want to see what is going to be installed check the [requirements.txt](./requirements.txt) file.

In summary two main tools will be installed and used in this notebook:

* **Llama Index**. Which will be used to download the dataset and to create the Semantic Index.
  > It is also possible to use **Azure AI Search** to create the semantic index. However, since it is going to be a small index, to simplify its creation in-memory we are going to use Llama Index.
* **RAGAS**. Ragas will be used to calculate the metrics for the Copilot that we are going to create.

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

# To create the RAG based copilot
from llama_index.core.llama_dataset import LabelledRagDataset
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, Settings
from ragas.metrics import (
    Faithfulness,
    ContextPrecision,
    ContextRecall
)

# To calculate the Generative AI quality metrics
from ragas.llms import LlamaIndexLLMWrapper
from ragas.embeddings import LlamaIndexEmbeddingsWrapper
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas.evaluation import evaluate
from ragas.run_config import RunConfig

# To create a simple PDF visualization tool
import pymupdf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# For formatting purposes
import textwrap

load_dotenv()

False

## Challenge 1 - Step 2: *Let's load the documents and test questions*

> This step takes around **1 minute** to complete.

The following code loads and prepares data from different sources (a JSON file and a directory of files) for further use:
1. The first line creates an instance of LabelledRagDataset by loading data from a JSON file `rag_dataset.json`. The `from_json` method is a class method that reads the JSON file, parses its content, and initializes the `LabelledRagDataset` object with the parsed data. This method ensures that the data is correctly formatted and validated before being used.
2. The second line initializes an instance of `SimpleDirectoryReader` with the directory path `source_files`. The `SimpleDirectoryReader` class is designed to read files from a specified directory. The `load_data` method is then called on this instance to load the documents from the directory. 

In [2]:
# explain the code below

rag_dataset = LabelledRagDataset.from_json("./data/rag_dataset.json")
documents = SimpleDirectoryReader(input_dir="./data/source_files/").load_data(
    show_progress=True
)

Loading files:   0%|                                                                                                                                                                                                       | 0/6 [00:00<?, ?file/s]

Loading files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.34s/file]


### Tool to visualize the reports just downloaded

The following cell creates a tool to visualize the PDF files we just downloaded. If you want to take a look at the downloaded reports manually, just navigate to the `data/source_files` folder.

In [3]:
data_source_path = "./data/source_files"

# List of pdf files just downloaded
pdf_files = [os.path.join(data_source_path, file_name) for file_name in os.listdir(data_source_path)]

# Function to render a specific page of a PDF
def render_pdf_page(pdf_path, page_number=0):
    # Open the PDF file
    pdf_document = pymupdf.open(pdf_path)
    
    # Ensure the page number is valid
    if page_number < 0 or page_number >= len(pdf_document):
        raise ValueError("Invalid page number.")
    
    # Get the page and render it as an image
    page = pdf_document[page_number]
    pix = page.get_pixmap()
    pdf_document.close()
    
    # Display the image using Matplotlib
    plt.figure(figsize=(10, 8))
    plt.imshow(pix.pil_image())
    plt.axis("off")
    plt.show()

# Function to update the displayed page
def update_page(step):
    global current_page
    pdf_document = pymupdf.open(dropdown.value)
    total_pages = len(pdf_document)
    pdf_document.close()
    
    # Update the current page index
    current_page += step
    if current_page < 0:
        current_page = 0
    elif current_page >= total_pages:
        current_page = total_pages - 1
    
    with output:
        output.clear_output()
        render_pdf_page(dropdown.value, current_page)

# Function to reset the viewer when a new PDF is selected
def reset_viewer(change):
    global current_page
    current_page = 0  # Reset to the first page
    with output:
        output.clear_output()
        render_pdf_page(dropdown.value, current_page)

# Create widgets
dropdown = widgets.Dropdown(
    options=pdf_files,
    description="Select a PDF:",
    style={"description_width": "initial"}
)

prev_button = widgets.Button(description="Previous Page")
next_button = widgets.Button(description="Next Page")
output = widgets.Output()

# Attach event listeners
prev_button.on_click(lambda _: update_page(-1))
next_button.on_click(lambda _: update_page(1))
dropdown.observe(reset_viewer, names="value")

# Initial display
reset_viewer(None)

# Display widgets and output
display(widgets.VBox([dropdown, widgets.HBox([prev_button, next_button]), output]))

## Challenge 1 - Step 3: *Let's create the semantic index*

> This process can take up to **15 seconds** to complete

A semantic index is a method used to organize and retrieve information based on the meaning and context of the content rather than just keywords. Unlike traditional keyword-based search systems, a semantic index understands the relationships between concepts and terms within the data, allowing for more accurate and relevant search results. In this context (i.e., ML and NLP), a semantic index leverages embeddings, which are numerical representations of words, phrases, or documents in a high-dimensional space. These embeddings capture the semantic meaning of the content, enabling the system to identify similar or related items even if they do not share exact keywords. The following code creates a semantic index using Azure OpenAI embeddings. The embedding model is used to generate embeddings for the documents, which are then indexed using `VectorStoreIndex` from `llamaIndex`. This process allows for efficient and meaningful retrieval of information based on the semantic content of the documents.

In [4]:
%%time
# Create the embedding model
embed_model = AzureOpenAIEmbedding(
    model='text-embedding-3-small', # Update with the embeddings deployment name
    api_key=os.environ['OPENAI_API_KEY'],
    api_version=os.environ['OPENAI_API_VERSION'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)

# Pass the embedding model to llamaIndex
Settings.embed_model = embed_model

# Create the actual index
index = VectorStoreIndex.from_documents(
    documents=documents,
    show_progress=True
)


Parsing nodes:   0%|          | 0/455 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

CPU times: user 1.41 s, sys: 139 ms, total: 1.55 s
Wall time: 15.6 s


## Challenge 1 - Step 4: *Let's create the copilot*

A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation, translation, summarization, and more. These models are characterized by their vast number of parameters, which enable them to understand and generate human-like text with high accuracy and fluency. LLMs are trained on extensive datasets using self-supervised learning, allowing them to acquire a deep understanding of language, including syntax, semantics, and contextual relationships.

Here we are creating an instance of a large language model using Azure OpenAI. The model, identified as `gpt-4o`, is configured with specific parameters such as `engine`, `model`, `temperature`, and API credentials. This setup allows the model to generate text based on the input it receives, making it a powerful tool for various natural language processing tasks.

The following code creates and configures the large language model using Azure OpenAI, and integrates it with the previously created semantic index to enable efficient and meaningful query processing.

In [5]:
# Create the Large Language Model
llm = AzureOpenAI(
    engine="gpt-4o", # Update with the language model deployment name 
    model="gpt-4o", # Update with the language model name
    temperature=0.0,
    api_key=os.environ['OPENAI_API_KEY'],
    api_version=os.environ['OPENAI_API_VERSION'],
    azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)

Settings.llm = llm

query_engine = index.as_query_engine() # this is the copilot

### Let's play with the copilot just created

In the test dataset, we have not only downloaded the source data (PDFs), but also 50 examples of questions/answer pairs to use with our Copilot. The following are some examples of the questions:

In [6]:
num_samples = 5

# Load some questions as example
samples = rag_dataset.to_pandas().loc[:num_samples-1, 'query'].values

display(Markdown("\n".join([ "* " + sample for sample in samples])))

* What is the objective of the Carbon Call initiative?
* What are the three key areas that will enable the scale of sustainability solutions?
* How many people in India did Microsoft provide water access to?
* How many people in Indonesia did Microsoft provide water access to?
* How many people in Brazil did Microsoft provide water access to?

The following is a simple tool that allows to ask questions to the new copilot

In [7]:
# Function to get a response
def get_response(user_input):
    return query_engine.query(user_input).response

# Interactive chatbot function
def chatbot():
    output = widgets.Output()
    scrollable_chat = widgets.Box(
        [output],
        layout=widgets.Layout(
            width="100%",
            height="300px",
            overflow="auto",
            border="1px solid #ccc",
            padding="5px",
            display="flex",
            flex_flow="column"

        )
    )
    # output = widgets.Output(layout={})
    text_box = widgets.Text(
        placeholder="Type your message here",
        description="You:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width="80%")
    )
    submit_button = widgets.Button(
        description="Send",
        button_style="primary",
        layout=widgets.Layout(width="10%")
    )

    chat_container = widgets.VBox([scrollable_chat, widgets.HBox([text_box, submit_button])])
    
    # Function to handle submission
    def on_submit(_):
        with output:

            user_message = text_box.value
            if user_message.strip():  # Process non-empty input
                display(widgets.HTML(
                    f"<div style='color: blue; font-weight: bold;'>You:</div> {text_box.value}"
                ))
                bot_response = get_response(user_message)
                display(widgets.HTML(
                    f"<div style='color: green; font-weight: bold;'>Bot:</div> {bot_response}"
                ))
            text_box.value = ""  # Clear the text box after submission
        
    # Attach event handlers
    submit_button.on_click(on_submit)
    
    display(chat_container)

# Run the chatbot
chatbot()

# Example question: What is Microsoft's most important contribution to the environment? → this is a question outside the dataset.

# Challenge 2: *Getting answers from our new Copilot*

## Challenge 2 - Step 1: *Let's take a look at the test dataset*

First, let's see an example from the test dataset that we downloaded together with the PDF files.

In [8]:
instance_idx = 0

def create_instance_md(k, v):
    md_str = f"**{k}:**\n\n"
    if k == "reference_contexts":
        return md_str + "\n".join([f"* {c}\n" for c in v])
    return md_str + f"{v}"

display(Markdown("### This is one instance in the dataset:\n\n"))
display(Markdown("\n\n".join([create_instance_md(k, v) for k,v in rag_dataset.to_pandas().iloc[instance_idx].items()])))

### This is one instance in the dataset:



**query:**

What is the objective of the Carbon Call initiative?

**reference_contexts:**

* In February 2022, Microsoft, ClimateWorks Foundation, and over 20 leading organizations launched an important new initiative called the Carbon Call. The objective of this program is to unify the world around a carbon accounting system that is more reliable and interoperable.


**reference_answer:**

The objective of Carbon Call is to unify the world around a carbon accounting system that is more reliable and interoperable

**reference_answer_by:**

human

**query_by:**

human

### The dataset
In total, the test dataset has 50 instances like the one detailed above, Let's take a look at some of them

In [9]:
rag_dataset.to_pandas().head(10)

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What is the objective of the Carbon Call initi...,"[In February 2022, Microsoft, ClimateWorks Fou...",The objective of Carbon Call is to unify the w...,human,human
1,What are the three key areas that will enable ...,[We will continue\ninvesting in three key area...,1 Advancing AI solutions for greater climate i...,human,human
2,How many people in India did Microsoft provide...,"[Water Table 3\n\nIndia: 309,921\nIndonesia: 2...","309,921",human,human
3,How many people in Indonesia did Microsoft pro...,"[Water Table 3\n\nIndia: 309,921\nIndonesia: 2...","225,389",human,human
4,How many people in Brazil did Microsoft provid...,"[Water Table 3\n\nIndia: 309,921\nIndonesia: 2...","16,408",human,human
5,How many people in Mexico did Microsoft provid...,"[Water Table 3\n\nIndia: 309,921\nIndonesia: 2...",340,human,human
6,How many more acres of land does Microsoft nee...,[Ecosystems Chart 1\nAchieving our target of p...,4998 more acres,human,human
7,What key insight was listed in page 56 of the ...,[25%\nWe are reducing idle\npower consumption\...,25%\nWe are reducing idle power consumption of...,human,human
8,What key insight was listed in page 61 of the ...,[100%\nOur key European\nand American\ndistrib...,100%\nOur key European and American distributi...,human,human
9,What are the four commitments listed on page 66?,[Our commitment\nUsing our voice on climate-re...,1. Using our voice on climate-related public p...,human,human


## Challenge 2 - Step 2: *Let's calculate the responses*

Now, let's have the model we created **provide answers to the questions** in the dataset. Also, to be able to calculate the quality metrics, we need to keep the **contexts retrieved**.  The retrieved contexts are the chunks of data retrieved from the semantic index to be used to answer the question.

> This process could take hours depending on the thoughtput of the LLM used. Just to keep the practical guide in a reasonable time, let's calculate the **responses** and **retrieved contexts** for a small sample of questions. For future steps, we will provide the full dataset with already pre-calculated answers.

In [10]:
# Select a subset sample to test
sample_size = 5
sub_dataset = LabelledRagDataset(examples=rag_dataset.examples[:sample_size])
sub_dataset.to_pandas()

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What is the objective of the Carbon Call initi...,"[In February 2022, Microsoft, ClimateWorks Fou...",The objective of Carbon Call is to unify the w...,human,human
1,What are the three key areas that will enable ...,[We will continue\ninvesting in three key area...,1 Advancing AI solutions for greater climate i...,human,human
2,How many people in India did Microsoft provide...,"[Water Table 3\n\nIndia: 309,921\nIndonesia: 2...","309,921",human,human
3,How many people in Indonesia did Microsoft pro...,"[Water Table 3\n\nIndia: 309,921\nIndonesia: 2...","225,389",human,human
4,How many people in Brazil did Microsoft provid...,"[Water Table 3\n\nIndia: 309,921\nIndonesia: 2...","16,408",human,human


Now, let's calculate the responses for the **subset** (this should take ~1m):

In [11]:
%%time
predictions = sub_dataset.make_predictions_with(
    predictor = query_engine,
    show_progress = True
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.43s/it]

CPU times: user 306 ms, sys: 18.1 ms, total: 325 ms
Wall time: 7.18 s


Let's look at the results:

> Here, since we are already preparing the data for the next challenge the column names changed:
> * `user_input` is the same `query` in the previous format.
> * `retrieved_contexts` is a list that includes the chucks of data used by our copilot to answer the question.
> * `response` is the `response` to the question created by our new copilot.
> * `reference` is the `reference answer` from our previous format and refers to the expected answer created by a human.

In [12]:
list_of_samples = []

for idx in range(len(sub_dataset.examples)):
    list_of_samples.append(
        SingleTurnSample (
            user_input = sub_dataset.examples[idx].query,
            reference = sub_dataset.examples[idx].reference_answer,
            response = predictions.predictions[idx].response,
            retrieved_contexts = predictions.predictions[idx].contexts
        )
    )

ragas_evaluation_dataset = EvaluationDataset(list_of_samples)
ragas_evaluation_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,What is the objective of the Carbon Call initi...,"[In 2022, we also released a preview version o...",The objective of the Carbon Call initiative is...,The objective of Carbon Call is to unify the w...
1,What are the three key areas that will enable ...,[What’s next \nScope 3 emissions reduction \nM...,The three key areas that will enable the scale...,1 Advancing AI solutions for greater climate i...
2,How many people in India did Microsoft provide...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...","Microsoft provided water access to 225,389 peo...","309,921"
3,How many people in Indonesia did Microsoft pro...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...",The specific number of people in Indonesia pro...,"225,389"
4,How many people in Brazil did Microsoft provid...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...","Microsoft provided water access to 16,408 peop...","16,408"


# Challenge 3: *Evaluate the quality of our new copilot*

Evaluating Generative AI (GenAI) solutions, especially those using Retrieval-Augmented Generation (RAG) frameworks, involves several key steps to ensure the quality and effectiveness of the generated outputs. 

GenAI solutions are evaluated using various metrics to measure the quality of the responses generated by the AI models. Libraries like RAGAS or promptflow are instrumental in this process. They help assess the solution's performance using metrics such as faithfulness, groundedness, context precision, and context recall1. These metrics provide insights into how accurate, relevant, and contextually appropriate the AI-generated responses are.

The evaluation process typically includes:

* **Model Selection and Building:** Choosing the right model and developing the GenAI application using iterative development practices and advanced prompt engineering techniques.
* **Quality Metrics:** Using metrics like faithfulness (accuracy and truthfulness), groundedness (relevance to the input context), context precision (alignment with the provided context), and context recall (inclusion of relevant information from the input context).
* **Explainability:** Adding explainability to these metrics to understand why certain responses are rated higher or lower. This helps in debugging and improving the overall solution.
* **User Feedback:** Collecting feedback from users to identify unexpected issues and improve the user experience.

For this demonstration, we use RAGAS, a specialized evaluation framework designed to assess the performance of RAG systems. It provides a structured approach to evaluate the effectiveness of RAG implementations by leveraging advanced Large Language Models (LLMs) as judges. 

## Challenge 3 - Step 1: *Initialize the LLM and Embedding models*

The first step is to initialize the LLM and Embedding models to be used to calculate the GenAI metrics:

In [13]:
evaluator_llm = LlamaIndexLLMWrapper(llm)
evaluator_embeddings = LlamaIndexEmbeddingsWrapper(embed_model)

## Challenge 3 - Step 2: *Calculate the GenAI metrics*

The process to calculate the GenAI metrics for all the questions in the test dataset could take hours depending on the throughtput of the LLM and the Embedding model used. To keep the time of this guide we are going to calculate only 3 metrics for the sub dataset created before:

* **Context precision:** Measures how much of the generated output is relevant and aligns with the context provided in the input.
* **Context recall:** Measures how much of the relevant information in the input context is included in the output.
* **Faithfulness:** Measures how accurate and truthful the generated output is in relation to the input context and factual correctness. Faithfulness is about avoiding "hallucinations" (made-up or false information).


> This process can take up to **25 seconds** to complete

In [14]:
%%time

metrics = [
    Faithfulness(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm)
]
ragas_evaluation_result = evaluate(
    dataset=ragas_evaluation_dataset,
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=evaluator_embeddings,
    run_config=RunConfig(timeout=1800, max_wait=180, max_retries=20),
    show_progress=True,
    batch_size=5
)

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Batch 1/3:   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 616 ms, sys: 46.4 ms, total: 663 ms
Wall time: 17.2 s


The following are the results of the calculation of the GenAI metrics

In [15]:
df_ragas_result = ragas_evaluation_result.to_pandas()
df_ragas_result

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall
0,What is the objective of the Carbon Call initi...,"[In 2022, we also released a preview version o...",The objective of the Carbon Call initiative is...,The objective of Carbon Call is to unify the w...,1.00,1.0,1.000000
1,What are the three key areas that will enable ...,[What’s next \nScope 3 emissions reduction \nM...,The three key areas that will enable the scale...,1 Advancing AI solutions for greater climate i...,0.75,0.5,0.666667
2,How many people in India did Microsoft provide...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...","Microsoft provided water access to 225,389 peo...","309,921",0.00,0.0,0.000000
3,How many people in Indonesia did Microsoft pro...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...",The specific number of people in Indonesia pro...,"225,389",1.00,0.0,0.000000
4,How many people in Brazil did Microsoft provid...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...","Microsoft provided water access to 16,408 peop...","16,408",0.00,1.0,0.000000


### Full dataset results

Let's take a look at a table with the responses, retrieved contexts and metrics already pre-calculated:

In [16]:
df_test_dataset = pd.read_json('./test-dataset.json', orient='records')
df_test_dataset.head(20)

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall
0,What is the objective of the Carbon Call initi...,"[In 2022, we also released a preview version o...",The objective of the Carbon Call initiative is...,The objective of Carbon Call is to unify the w...,1.000000,1.0,1.000000
1,What are the three key areas that will enable ...,[What’s next \nScope 3 emissions reduction \nM...,The three key areas that will enable the scale...,1 Advancing AI solutions for greater climate i...,0.750000,0.0,0.666667
2,How many people in India did Microsoft provide...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...","Microsoft provided water access to 225,389 peo...","309,921",0.000000,0.0,0.000000
3,How many people in Indonesia did Microsoft pro...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...",The specific number of people in Indonesia pro...,"225,389",1.000000,1.0,0.000000
4,How many people in Brazil did Microsoft provid...,"[Chile\nSouth Africa\n16,408 \nBrazil\n340 \n...","Microsoft provided water access to 552,058 peo...","16,408",1.000000,1.0,0.000000
5,How many people in Mexico did Microsoft provid...,[Improving access \nto water \nWe continue to ...,"Microsoft provided water access to 309,921 peo...",340,0.000000,0.0,1.000000
6,How many more acres of land does Microsoft nee...,[• • \na \n47\n | | | \nCommitments and progre...,"According to Ecosystems Chart 1, Microsoft has...",4998 more acres,0.800000,1.0,1.000000
7,What key insight was listed in page 56 of the ...,[Leadership About Highlights How We Operate Wh...,The context provided does not include informat...,25%\nWe are reducing idle power consumption of...,1.000000,0.0,0.000000
8,What key insight was listed in page 61 of the ...,[Leadership About Highlights How We Operate Wh...,The context provided does not include informat...,100%\nOur key European and American distributi...,1.000000,0.0,0.000000
9,What are the four commitments listed on page 66?,[UN Sustainable Development Goal Apple’s suppo...,The four commitments listed on page 66 are: \n...,1. Using our voice on climate-related public p...,0.250000,0.0,0.000000
